In [35]:
import os
import glob
import geopandas as gpd
from shapely.ops import unary_union
import pandas as pd
import numpy as np
import jupyter_black

In [96]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = None
jupyter_black.load()

In [4]:
# LA neighborhoods
hoods_gdf = gpd.read_file('https://stilesdata.com/gis/la_city_hoods_county_munis.geojson')

In [74]:
# The trees have cities and LA city neighhoods in the 'name' column. Format LA neighborhoods as "Los Angeles (Neighborhood)"
hoods_gdf["place"] = np.where(
    hoods_gdf["city"] == "los-angeles",
    "Los Angeles ("
    + hoods_gdf["name"]
    + ")",  # LA trees get formatted neighborhood label
    hoods_gdf["name"],  # Non-LA trees keep their city name from 'name'
)

In [30]:
# Read the 'trees' layer from the GeoPackage
geo_path = "../data/large/combined_trees.gpkg"
gdf = gpd.read_file(geo_path, layer="trees")
gdf.columns = gdf.columns.str.lower()

In [31]:
gdf.head()

,name,height_m,city,geometry
0,Shadow Hills,5,los-angeles,POINT (-118.36483 34.23694)
1,Shadow Hills,20,los-angeles,POINT (-118.36263 34.23697)
2,Shadow Hills,5,los-angeles,POINT (-118.35676 34.23703)
3,Shadow Hills,5,los-angeles,POINT (-118.3566 34.23703)
4,Shadow Hills,4,los-angeles,POINT (-118.35352 34.23707)


In [32]:
# The trees have cities and LA city neighhoods in the 'name' column. Format LA neighborhoods as "Los Angeles (Neighborhood)"
gdf["place"] = np.where(
    gdf["city"] == "los-angeles",
    "Los Angeles (" + gdf["name"] + ")",  # LA trees get formatted neighborhood label
    gdf["name"],  # Non-LA trees keep their city name from 'name'
)

In [39]:
# What's the extent of the lidar coverage?
tree_coverage_boundary = gdf.unary_union.convex_hull

In [75]:
# Choose a projected CRS suitable for LA (California Albers)
proj_crs = "EPSG:3310"

# Reproject neighborhoods and tree boundary for accurate area calculations
hoods_proj = hoods_gdf.to_crs(proj_crs)
tree_boundary_proj = (
    gpd.GeoSeries([tree_coverage_boundary], crs=gdf.crs).to_crs(proj_crs).iloc[0]
)

# Calculate intersection and total areas in square meters
hoods_proj["intersection_area"] = hoods_proj.geometry.intersection(
    tree_boundary_proj
).area
hoods_proj["total_area"] = hoods_proj.geometry.area

# Compute the share of each neighborhood covered by lidar
hoods_proj["coverage_share"] = round(
    (hoods_proj["intersection_area"] / hoods_proj["total_area"]) * 100, 2
)

In [98]:
# Define conversion: 1 sq meter = 1/2,589,988.110336 sq miles
conversion_factor = 1 / 2589988.110336
# Compute area in square miles for each neighborhood
hoods_proj["area_sq_miles"] = hoods_proj["total_area"] * conversion_factor

In [105]:
hoods_w_full_tree_coverage = (
    hoods_proj.query("coverage_share == 100").sort_values("place").copy()
)

In [112]:
hoods_w_full_tree_coverage_list = hoods_w_full_tree_coverage["place"].to_list()

In [107]:
trees_w_hoods_gdf = gdf.query(f"place.isin({hoods_w_full_tree_coverage_list})").copy()

In [152]:
hoods_lookup = hoods_proj.set_index("place")["area_sq_miles"].round(2).to_dict()

place_tree_counts = (
    trees_w_hoods_gdf.groupby("place")
    .agg({"name": "count"})
    .reset_index()
    .rename(columns={"name": "count"})
    .sort_values("count", ascending=False)
).reset_index(drop=True)

place_tree_counts["area_sq_miles"] = place_tree_counts["place"].map(hoods_lookup)

place_tree_counts["trees_per_sq_mile"] = round(
    place_tree_counts["count"] / place_tree_counts["area_sq_miles"], 2
)

In [153]:
place_tree_counts = place_tree_counts.sort_values("trees_per_sq_mile", ascending=False)

In [156]:
density_lookup = (
    place_tree_counts.set_index("place")["trees_per_sq_mile"].round(2).to_dict()
)

In [158]:
hoods_gdf["trees_per_sqmi"] = hoods_gdf["place"].map(density_lookup)

Index(['name', 'slug', 'county', 'type', 'city', 'region', 'geometry',
       'intersection_area', 'total_area', 'coverage_share', 'place',
       'trees_per_sqmi'],
      dtype='object')